In [ ]:
# importing the libraries
import pandas as pd
import numpy as np
import math
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# PyTorch libraries and modules
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout


from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


# self write tools
from tools import LABEL_ENCODER, transforms_train, transforms_test
from models import CNN, RNN, NN
from dataset import CnnDataset, TestDataset, CREATE_CNN_TRAIN_ITERATOR, CREATE_CNN_VALID_ITERATOR, words, TEST_DATASET

# Check GPU

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

# Global Variables

In [ ]:

BATCH_SIZE = 512
EPOCHS = 100





# 2. Model

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (image, (description, l), features, y) in iterator:

        features = features.to(device, dtype=torch.float)

  

        y = y.to(device, dtype=torch.int64)
        
        optimizer.zero_grad()
                
        y_pred = model(features)
        
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_cnn(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for (image, (description, l), features, y) in iterator:

            features = features.to(device, dtype=torch.float)

            y = y.to(device, dtype=torch.int64)
                    
            y_pred = model(features)
            
            loss = criterion(y_pred, y)
            
            acc = calculate_accuracy(y_pred, y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [ ]:
cnn_train_iterator = CREATE_CNN_TRAIN_ITERATOR(transforms_train, BATCH_SIZE)
cnn_valid_iterator = CREATE_CNN_VALID_ITERATOR(BATCH_SIZE)

In [ ]:
criterion = CrossEntropyLoss()
criterion = criterion.cuda()

model = NN(input_size=4, hidden_size=15, output_size=27).cuda()

optimizer = Adam(model.parameters(), lr=0.001)








In [ ]:




train_accuracy_list = []
train_loss_list = []
valid_acc_list = []
valid_loss_list = []
best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    # train
    train_loss, train_acc = train(model, cnn_train_iterator, optimizer, criterion, device)
    
    # valid
    valid_loss, valid_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

    # save best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'output/linear-model.pt')

    # Track the accuracy
    train_accuracy_list.append(train_acc)
    train_loss_list.append(train_loss)
    valid_acc_list.append(valid_acc)
    valid_loss_list.append(valid_loss)
        
    # print epoch info
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        
 



In [ ]:
model.load_state_dict(torch.load('output/linear-model.pt'))
test_loss, test_acc = evaluate_cnn(model, cnn_valid_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')